# Faces

In [3]:
import matplotlib.pyplot as plt 
import numpy as np

# Create data for training

In [ ]:
from typing import List
import math


def create_binary_list_from_int(number: int) -> List[int]:
    if number < 0 or type(number) is not int:
        raise ValueError("Only Positive integers are allowed")

    return [int(x) for x in list(bin(number))[2:]]


max_value = 200
n_samples = 50000

input_length = int(math.log(max_value, 2)) + 1
sampled_integers = np.random.randint(0, int(max_value / 2), n_samples)

# Generate a list of binary numbers for training.
data = [create_binary_list_from_int(int(x * 2)) for x in sampled_integers]

data = [([0] * (input_length - len(x))) + x for x in data]

data = np.array(data)

# Define Discriminator & Generator

## Layer's Architecture

In [1]:
import torch
import torch.nn as nn


class Generator(nn.Module):
    def __init__(self, input_length: int):
        super(Generator, self).__init__()
        self.dense_layer = nn.Linear(int(input_length), int(input_length))
        self.activation = nn.Sigmoid()

    def forward(self, x):
        return self.activation(self.dense_layer(x))


class Discriminator(nn.Module):
    def __init__(self, input_length: int):
        super(Discriminator, self).__init__()
        self.dense = nn.Linear(int(input_length), 1);
        self.activation = nn.Sigmoid()

    def forward(self, x):
        return self.activation(self.dense(x))
    

## Optimizers and loss functions

In [ ]:
generator = Generator(input_length)
discriminator = Discriminator(input_length)
loss = nn.BCELoss()
generator_optimizer = torch.optim.Adam(generator.parameters(), lr=0.001)
discriminator_optimizer = torch.optim.Adam(discriminator.parameters(), lr=0.001)

# Train GAN

In [ ]:
from trainers.general_trainer import Trainer

trainer = Trainer(generator, discriminator, loss, loss, generator_optimizer, discriminator_optimizer)
trainer.train_general(data, input_length, 16, 1000)

In [ ]:
noise = torch.randint(0, 2, size=(10, input_length)).float()
fake_data = trainer.generator(noise)

data = fake_data[0]
boolean_tensor = (fake_data>0.5).to(torch.int)
valor_entero = np.packbits(boolean_tensor)
print(valor_entero)

In [ ]:
batch_size = 16
image_height = 128
image_width = 128
num_channels = 3

# Generate random input data
random_input = torch.randn(batch_size, num_channels, image_height, image_width)

# Print the shape of the random input data
print(random_input.shape)


In [ ]:
noise = torch.tensor(np.random.normal(0, 1, (4, 10)), dtype=torch.float)
print(noise.size(0))

In [ ]:
class SimpleRegressionModel(nn.Module):
    def __init__(self):
        super(SimpleRegressionModel, self).__init__()
        self.features = nn.Sequential(
            nn.Conv2d(3, 64, kernel_size=3, stride=1, padding=1),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=2, stride=2),
            nn.Conv2d(64, 128, kernel_size=3, stride=1, padding=1),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=2, stride=2)
        )
        self.fc = nn.Sequential(
            nn.Linear(128 * 32 * 32, 256),  # Adjust input size based on your image dimensions
            nn.ReLU(inplace=True),
            nn.Linear(256, 1)  # Output layer with one value
        )

    def forward(self, x):
        x = self.features(x)
        print(x.shape)
        x = x.view(x.size(0), -1)  # Flatten the tensor
        x = self.fc(x)
        return x

# Create an instance of the model
model = SimpleRegressionModel()


model(random_input)